## Designing a segmentation workflow (Multichannel)

#### This notebook illustrates how to design a segmentation workflow, so a different threshold or operation is used based on the channel.

Define code for segmentation based on [Interactive Workflows](https://github.com/BioimageAnalysisCoreWEHI/napari_lattice/wiki/5.-Workflows-(Interactive:-no-coding)#interactive). However, lets use a different operation for each channel.

* Median filter (1,1,0) regardless of the channel
* Triangle Threshold for channel 1, and 
* Otsu Threshold for channel 2
* Connected components labelling

TODO: Add example for multichannel segmentation


In [5]:
#The custom python function will look like:

#measure_regionprops_multich.py

from napari_lattice import config
from skimage.filters import threshold_triangle, threshold_otsu

def segment_multich(img):
    #if first channel, use threshold_triangle
    if config.channel == 0:
        binary_img = threshold_triangle(img)
    #if second channel, use Otsu threshold
    elif config.channel == 1:
        binary_img = threshold_otsu(img)
    return binary_img

    

Above is saved in a file called:  `measure_regionprops_multich.py`

In [9]:
#import workflow
from napari_workflows import Workflow
import measure_regionprops_multich
import pyclesperanto_prototype as cle

#We initialise a workflow
segmentation_multich_workflow = Workflow()

#For each workflow , we need to define the name of the task, the function name and the function parameters
#For the input image, we only need to pass a variable or we pass the name of the previous workflow task_name


#Set task for median first 
#you can use any name for input image
input_image = "input_img"

#To set a task, we use the set method
segmentation_multich_workflow.set("median",
                          cle.median_sphere,
                          source = input_image,
                          radius_x = 1, 
                          radius_y= 1,
                          radius_z= 1)

#The second task will use input from above. so set the task name 'median' as input 
segmentation_multich_workflow.set("binarise_multich",
                          measure_regionprops_multich.segment_multich,
                          img = "gaussian")

segmentation_multich_workflow.set("label",
                          cle.connected_components_labeling_box,
                          binary_input = "binarise_multich")

#Printing a workflow will show you the arguments
print(segmentation_multich_workflow)

Workflow:
median <- (<function median_sphere at 0x000002BFF4859AF0>, 'input_img', None, 1, 1, 1)
binarise_multich <- (<function segment_multich at 0x000002BF985B60D0>, 'gaussian')
label <- (<function connected_components_labeling_box at 0x000002BFF4908B80>, 'binarise_multich')



In [8]:
from napari_workflows import _io_yaml_v1

_io_yaml_v1.save_workflow("regionprops_multich_workflow.yml", segmentation_multich_workflow)

To use this in `napari-lattice`, import it in under the `Workflow` tab.
You can try `Preview Workflow` to test it on a region or `Apply Workflow and Save` to apply it across a range of timepoints/channels.

Refer to the `napari-lattice Wiki` for more details